<a href="https://colab.research.google.com/github/solomontessema/Agentic-AI-with-Python/blob/main/notebooks/Data%20Connectivity%20and%20Email%20Services/Natural_Language_to_SQL_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Natural Language to SQL Generator</h1></td>
  </tr>
</table>

In [1]:
!pip install -qU langchain langchain-openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.5 MB/s eta 0:00:00


In [3]:
import sqlite3

class SQLiteConnector:
    def __init__(self, db_path="data.db"):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()

    def list_tables(self):
        self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        return [row[0] for row in self.cursor.fetchall()]

    def describe_table(self, table_name):
        self.cursor.execute(f"PRAGMA table_info({table_name});")
        return self.cursor.fetchall()

    def get_schema_summary(self):
        summary = []
        for table in self.list_tables():
            cols = self.describe_table(table)
            formatted = f"Table: {table} Columns: {[col[1] for col in cols]}"
            summary.append(formatted)
        return "\n".join(summary)


In [ ]:
import sys
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

connector = SQLiteConnector()
schema_summary = connector.get_schema_summary()

llm = ChatOpenAI(model="gpt-4", temperature=0)

prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
    Given the database schema:
    {schema}

    Write only the SQLite SQL query to answer the question below.
    Do not include any explanation or commentary.

    Question:
    {question}
    """
)

sql_runnable = prompt | llm | StrOutputParser()

def generate_sql(question: str):
    return sql_runnable.run({"schema": schema_summary, "question": question})

def run_query(query):
    try:
        cursor = connector.conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return columns, rows
    except Exception as e:
        return None, f"SQL Error: {str(e)}"


def format_results(columns, rows):
    if not rows:
        return "No results found."

    output = ""
    for row in rows:
        row_text = ", ".join(f"{col}: {val}" for col, val in zip(columns, row))
        output += f"- {row_text}\n"
    return output


def execute_and_respond(question):
    sql = generate_sql(question)
    columns, rows_or_error = run_query(sql)

    if isinstance(rows_or_error, str):  # error message
        return f"Error executing SQL: {rows_or_error}"
    return format_results(columns, rows_or_error)



In [ ]:
print(generate_sql("How many users signed up in 2023?"))
response = execute_and_respond("How many users signed up in 2023?")
print(response)